# Contacts between SARS-CoV-2 and ACE2

To better understand the complex between the RBD and ACE2, we did some analysis of contacts between RBD and ACE2. Using the Bio.PDB Python library, I was able to find all contacts between the SARS-CoV-2 RBD and ACE2 using the 6M0J structure. For our purposes, if a residue C Alpha atom is within 7 Angstroms of another residue of C Alpha atom, the two residues are considered in contact. Results showed that there are 16 contacts.

In [67]:
from tabulate import tabulate
import numpy as np
from Bio.PDB import *

In [69]:
parser = PDBParser(PERMISSIVE=True, QUIET=True)
data = parser.get_structure("6M0J", "6M0J.pdb")
model = data.get_models()

models = list(model)
chains = list(models[0].get_chains())
residuesA = list(chains[0].get_residues())
residuesE = list(chains[1].get_residues())
# residue identifier, atom
cAlphaA = [[], []]
cAlphaE = [[], []]
for i in range(len(residuesA)):
    atoms = list(residuesA[i].get_atoms())
    for j in range(len(atoms)):
        if atoms[j].get_id() == "CA":
            cAlphaA[0].append(residuesA[i].get_resname() + str(residuesA[i].get_id()[1]))
            cAlphaA[1].append(atoms[j])
            break
for i in range(len(residuesE)):
    atoms = list(residuesE[i].get_atoms())
    for j in range(len(atoms)):
        if atoms[j].get_id() == "CA":
            cAlphaE[0].append(residuesE[i].get_resname() + str(residuesE[i].get_id()[1]))
            cAlphaE[1].append(atoms[j])
            break

# RBD residue, count contacts with ACE2, list of contacted residues
data = [[], [], []]

countTot = 0
for i in range(len(cAlphaE[0])):
    count = 0
    contacts = ""
    for j in range(len(cAlphaA[0])):
        if(cAlphaE[1][i] - cAlphaA[1][j]) <= 7:
            count += 1
            countTot += 1
            contacts += cAlphaA[0][j] + " " + str(int((cAlphaE[1][i] - cAlphaA[1][j])*100)/100) + "\n"
    contacts = contacts[:len(contacts)-2]
    if count > 0:
        data[0].append(cAlphaE[0][i])
        data[1].append(count)
        data[2].append(contacts)

aceRes = dict()
for i in range(len(data[2])):
    res = data[2][i].split("\n")
    for s in res:
        if s.split(" ")[0] in aceRes:
            aceRes[s.split(" ")[0]] += 1
        else:
            aceRes[s.split(" ")[0]] = 1

dataInRows = []
for i in range(len(data[0])):
    arr = [str(data[0][i]) + "(" + str(data[1][i]) + ")", data[2][i]]
    dataInRows.append(arr)

In [70]:
print("Total Contacts: " + str(countTot))
print("Cutoff Distance: 7 Angstroms")
print()
print(str(len(data[0])) + " RBD Contact Residues:", end=" ")
for i in range(len(data[0])):
    print(str(data[0][i]) + "(" + str(data[1][i]) + ")", end=" ")
print()
print(str(len(aceRes)) + " ACE2 Contact Residues:", end = " ")
for c in aceRes:
    print(str(c.split(" ")[0]) + "(" + str(aceRes[c]) + ")", end=" ")
print()
print()
print(tabulate((dataInRows), headers=["RBD Residue", "ACE2 Residues"]))

Total Contacts: 16
Cutoff Distance: 7 Angstroms

8 RBD Contact Residues: ALA475(1) GLY476(2) GLY496(1) THR500(2) ASN501(3) GLY502(4) GLY504(1) TYR505(2) 
7 ACE2 Contact Residues: GLN24(2) SER19(1) ASP38(1) GLY326(2) ASP355(3) LYS353(3) GLY354(4) 

RBD Residue    ACE2 Residues
-------------  ---------------
ALA475(1)      GLN24 6.1
GLY476(2)      SER19 6.17
               GLN24 6.4
GLY496(1)      ASP38 6.7
THR500(2)      GLY326 6.58
               ASP355 6.6
ASN501(3)      LYS353 5.5
               GLY354 5.81
               ASP355 5.2
GLY502(4)      GLY326 6.55
               LYS353 6.03
               GLY354 4.11
               ASP355 5.1
GLY504(1)      GLY354 6.7
TYR505(2)      LYS353 5.74
               GLY354 6.6
